# Arctic ice application

**Goal:** "promote" the persistence of an $H_1$ feature by pixel weighting to find patterns in the ice data.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ripser import ripser
from H1_optimizer import one_step_reweight_enhance_H1
from loadsir import loadsir
from loadsir import loadsir
import os
from sklearn.decomposition import PCA

### Plot all persistence diagrams

In [ ]:
# Point this to the full ice data if you have it. Otherwise, skip this and proceed below with 2006 data.
folder_prefix = '.../daacdata.apps.nsidc.org/pub/DATASETS/nsidc0265_QuikSCAT_ice_extent/data/north/binary/eggs'

def reduce_image(img, scale=30):
    rows, cols = int(img.shape[0]/scale), int(img.shape[1]/scale)
    rimg = np.zeros((rows, cols))
    for i in range(rows):
        for j in range(cols):
            rimg[i,j] = np.sum(img[i*scale:(i+1)*scale, j*scale:(j+1)*scale])
    return rimg

all_eggs = []
for year in range(1999, 2010):
    eggs = []
    days = []
    for file in tqdm(sorted(os.listdir('{}/{}'.format(folder_prefix, year)))):
        #print('ice/{}/'.format(year) + file)
        sir = loadsir('{}/{}/'.format(folder_prefix, year) + file)
        binary_img = np.array(np.abs(np.abs(sir[0] - 1) < 1e-2), dtype= int)
        eggs.append(reduce_image(binary_img))
        days.append(file.split('.')[0][-3:])

    X = np.array([e.ravel() for e in eggs])
    X = X/X.max()
    all_eggs.append(X)

In [ ]:
for year, X in enumerate(all_eggs):
    PD = ripser(X, maxdim=1, metric='manhattan')['dgms'][1]
    plt.scatter(PD[:,0], PD[:,1])
    plt.plot([0,110],[0,110], 'k--')
    plt.xlim([0,110])
    plt.ylim([0,110])
    plt.gca().set_aspect('equal', 'box')
    plt.xlabel('birth')
    plt.ylabel('death')
    plt.title('{}'.format(year+1999))
    plt.savefig('ice_eggs_{}_PD.png'.format(year+1999), bbox_inches='tight', dpi=300)
    plt.close()

### Pixel weighting on the 2006 data

This dataset is provided in this repo.

In [ ]:
folder_prefix = 'ice/'

def reduce_image(img, scale=30):
    rows, cols = int(img.shape[0]/scale), int(img.shape[1]/scale)
    rimg = np.zeros((rows, cols))
    for i in range(rows):
        for j in range(cols):
            rimg[i,j] = np.sum(img[i*scale:(i+1)*scale, j*scale:(j+1)*scale])
    return rimg

year = 2006
eggs = []
days = []
for file in tqdm(sorted(os.listdir('{}/{}'.format(folder_prefix, year)))):
    #print('ice/{}/'.format(year) + file)
    sir = loadsir('{}/{}/'.format(folder_prefix, year) + file)
    binary_img = np.array(np.abs(np.abs(sir[0] - 1) < 1e-2), dtype= int)
    eggs.append(reduce_image(binary_img))
    days.append(file.split('.')[0][-3:])

X = np.array([e.ravel() for e in eggs])
X = X/X.max()

In [ ]:
fig, ax = plt.subplots(8,8, figsize=(10,10))
idxs = np.linspace(0, len(eggs)-1, 64, dtype=int)
for i in range(8):
    for j in range(8):
        ax[i,j].axis('off')
        ax[i,j].imshow(eggs[idxs[i*8+j]], cmap='gray')

plt.savefig('ice_eggs_2006.png', bbox_inches='tight', dpi=300)

In [ ]:
T = one_step_reweight_enhance_H1(X, method='cochains', epsilon=[0.01, 0.05, 0.1])


plt.imshow(T.reshape(eggs[0].shape), cmap='viridis')
plt.colorbar()
plt.savefig('ice_eggs_2006_weights.png', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
os.makedirs('arctic', exist_ok=True)
pca = PCA(n_components=2)
A = pca.fit_transform(X)
loadings = pca.components_.T
plt.scatter(A[:,0], A[:,1], s=5, c=range(A.shape[0]))
plt.colorbar()
plt.axis('equal')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.savefig('ice_eggs_2006_PCA_uniform.png', bbox_inches='tight', dpi=300)
plt.show()
limit = np.max(np.abs(loadings))
plt.imshow(loadings[:,0].reshape(eggs[0].shape), cmap='PiYG', vmin=-limit, vmax=limit)
plt.savefig('ice_eggs_2006_PC1_uniform.png', bbox_inches='tight', dpi=300)
plt.show()
plt.imshow(loadings[:,1].reshape(eggs[0].shape), cmap='PiYG', vmin=-limit, vmax=limit)
plt.colorbar()
plt.savefig('ice_eggs_2006_PC2_uniform.png', bbox_inches='tight', dpi=300)
plt.show()

pca = PCA(n_components=2)
A = pca.fit_transform(T*X)
loadings = pca.components_.T
plt.scatter(A[:,0], A[:,1], s=5, c=range(A.shape[0]))
plt.colorbar()
plt.axis('equal')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.savefig('ice_eggs_2006_PCA_learned.png', bbox_inches='tight', dpi=300)
plt.show()
limit = np.max(np.abs(loadings))
plt.imshow(loadings[:,0].reshape(eggs[0].shape), cmap='PiYG', vmin=-limit, vmax=limit)
plt.title('PC1')
plt.savefig('ice_eggs_2006_PC1_learned.png', bbox_inches='tight', dpi=300)
plt.show()
plt.imshow(loadings[:,1].reshape(eggs[0].shape), cmap='PiYG', vmin=-limit, vmax=limit)
plt.title('PC2')
plt.colorbar()
plt.savefig('ice_eggs_2006_PC2_learned.png', bbox_inches='tight', dpi=300)
plt.show()